## LSTM LM

Train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

See [link](https://www.tensorflow.org/programmers_guide/embedding) for TF Embedddings

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

In [1]:
import os
import numpy as np
import tensorflow as tf
import zipfile
import string
import random

In [2]:
filename = 'text8.zip'
def read_text8Zip(filename):
    with zipfile.ZipFile(filename) as f:
        # Sometimes has a file list in one archieve (zip) file.
        name = f.namelist()[0] # we only need the first file in this case
        # compat stands for compatibility, solving versions of python 2 and 3
        data = tf.compat.as_str(f.read(name))
    return data

text = read_text8Zip(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [3]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [4]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter_id = ord(string.ascii_lowercase[0])

def char2id(char):
    if char in string.ascii_lowercase:
        return ord(char) - first_letter_id + 1
    elif char == ' ':
        return 0
    else:
        print('Unexpected character: %s' % char)
        return 0
    
def id2char(id):
    if id > 0:
        return chr(id - 1 + first_letter_id)
    else:
        return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [5]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // batch_size
        self._cursor = [ offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()
        
    def _next_batch(self):
        # Generate a single batch from the current cursor position in the data.
        # Then move one step of the cursor
        batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
        for b in np.arange(self._batch_size):
            batch[b, char2id(self._text[self._cursor[b]])] = 1.0
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch
    
    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

def characters(list_of_pdfs):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""
    return [id2char(id) for id in np.argmax(list_of_pdfs,axis=1)]

def batches2string(batches):
    """Convert a sequence of batches back into their (most likely) string
    representation."""
    # Very Elegant!!
    s = [''] * batches[0].shape[0] # batch_size
    for b in batches:
        s = [''.join(x) for x in zip(s, characters(b))]
    return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
# [bigram]:
valid_batches = BatchGenerator(valid_text, 1, 2)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [6]:
def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
    predictions[predictions < 1e-10] = 1e-10
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
    """Sample one element from a distribution assumed to be an array of normalized
    probabilities.
    """
    r = random.uniform(0, 1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
            return i
    return len(distribution) - 1
    
def sample(prediction):
    """Turn a (column) prediction into 1-hot encoded samples."""
    p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
    p[0, sample_distribution(prediction[0])] = 1.0
    return p

def random_distribution():
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
    return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [7]:
embedding_dim = 128
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
    # ===== Variables Definitions =====
    
    # [bigram]: Embeddings Variables
    embedding_mat = tf.Variable(tf.truncated_normal([pow(vocabulary_size,2), embedding_dim], -0.1, 0.1))
    
    # Input gate: input, previous output, and bias.
    # decides which values we’ll update.
    ix = tf.Variable(tf.truncated_normal([embedding_dim, num_nodes], -0.1, 0.1))
    im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ib = tf.Variable(tf.zeros([1, num_nodes]))
    
    # Forget gate: input, previous output, and bias.
    # decide what information we’re going to throw away from the cell state
    fx = tf.Variable(tf.truncated_normal([embedding_dim, num_nodes], -0.1, 0.1))
    fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    fb = tf.Variable(tf.zeros([1, num_nodes]))
    
    # Memory cell: input, state and bias.
    # creates a vector of new candidate values, C~t, that could be added to the state
    cx = tf.Variable(tf.truncated_normal([embedding_dim, num_nodes], -0.1, 0.1))
    cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    cb = tf.Variable(tf.zeros([1, num_nodes]))
    
    # Output gate: input, previous output, and bias.
    # decides what parts of the cell state we’re going to output
    ox = tf.Variable(tf.truncated_normal([embedding_dim, num_nodes], -0.1, 0.1))
    om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ob = tf.Variable(tf.zeros([1, num_nodes]))
    
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))
    
    # ===== Define the Input Tensor =====
    # Input data.
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
    train_inputs = zip(train_data[:num_unrollings-1],train_data[1:num_unrollings]) # [bigram]: zip the bi-gram
    # [bigram]: Since we use bi-gram, start from 2
    train_labels = train_data[2:]  # labels are inputs shifted by one time step.
    
    # ===== Graph Construction =====
    # exactly the same as colah's blog
    # http://colah.github.io/posts/2015-08-Understanding-LSTMs/
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""        
        input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
        forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
        update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
        return output_gate * tf.tanh(state), state
    
    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        # [bigram]: get bi-gram index and lookup embedding variables
        idx = tf.argmax(i[0],axis=1) + vocabulary_size * tf.argmax(i[1],axis=1)
        i_embed = tf.nn.embedding_lookup(embedding_mat, idx)
        output, state = lstm_cell(i_embed, output, state)
        outputs.append(output)
        
    # ===== Define Loss and Optimizer =====
    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output),saved_state.assign(state)]):
        # Classifier.
        # tf.concat: Concatenates the list of tensors along dimension axis
        # https://www.tensorflow.org/api_docs/python/tf/concat
        logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
        # softmax_cross_entropy_with_logits: logits and labels must have the same shape, e.g. [batch_size, num_classes]
        # labels: Each row labels[i] must be a valid probability distribution.
        # logits: Unscaled log probabilities.
        # https://www.tensorflow.org/api_docs/python/tf/nn/softmax_cross_entropy_with_logits
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf.concat(train_labels, 0), logits=logits))
        
    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)
    
    # Predictions.
    train_prediction = tf.nn.softmax(logits)
    
    # Sampling and validation eval: batch 1, no unrolling.
    # [bigram]:
    sample_input = (tf.placeholder(tf.float32, shape=[1, vocabulary_size]), tf.placeholder(tf.float32, shape=[1, vocabulary_size]))
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(saved_sample_output.assign(tf.zeros([1, num_nodes])),saved_sample_state.assign(tf.zeros([1, num_nodes])))
    # get the embeddings
    idx_sample = tf.argmax(sample_input[0],axis=1) + vocabulary_size * tf.argmax(sample_input[1],axis=1)
    sample_embed = tf.nn.embedding_lookup(embedding_mat, idx_sample)
    sample_output, sample_state = lstm_cell(sample_embed, saved_sample_output, saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b)) 

In [8]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print('Init')
    mean_loss = 0
    for step in np.arange(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i]
            
        _, l, predictions, lr = sess.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            # [bigram]:
            labels = np.concatenate(list(batches)[2:])
            print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
            
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    # [bigram]:
                    feed = (sample(random_distribution()), sample(random_distribution()))
                    #print('len(feed)=%d' % len(feed))
                    #print('feed[0].shape={}'.format(feed[0].shape))
                    sentence = characters(feed[0])[0]+characters(feed[1])[0]
                    #sentence = [characters(feed)[0] for _ in np.arange(2)]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: feed})
                        # [bigram]:
                        feed = (feed[1], sample(prediction))
                        sentence += characters(feed[1])[0]
                    print(sentence)
                print('=' * 80)
            
            # Measure validation set perplexity.
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                # [bigram]:
                predictions = sample_prediction.eval({sample_input: (b[0],b[1])})
                valid_logprob = valid_logprob + logprob(predictions, b[2])
            print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))

Init
Average loss at step 0: 3.358214 learning rate: 10.000000
Minibatch perplexity: 28.74
bi a   t r d k o c m s a r a   o   m f o p i f t j   d e t c d k r o h h w c m s 
tl h y v c v a i n i t   i   e w   d r u k a g m i s   h v m o f j m u x p g o l 
ky p   h x o i z   t w s f a r n a m x i w f     n h e t   q k c t e d d o     q 
rm   l   a t v g e r t eeh m o v w o mes a j r v w b s f t e w v b v i d r e r e 
ro n d q n j z   g   b m vdm e m a x n d s b   k e f p e r e y m i v z l r n i i 
Validation set perplexity: 227.83
Average loss at step 100: 2.791864 learning rate: 10.000000
Minibatch perplexity: 10.82
Validation set perplexity: 11.65
Average loss at step 200: 2.238934 learning rate: 10.000000
Minibatch perplexity: 8.79
Validation set perplexity: 9.36
Average loss at step 300: 2.092189 learning rate: 10.000000
Minibatch perplexity: 7.91
Validation set perplexity: 9.08
Average loss at step 400: 1.996270 learning rate: 10.000000
Minibatch perplexity: 7.66
Validation set perp

Validation set perplexity: 6.94
Average loss at step 4500: 1.624910 learning rate: 10.000000
Minibatch perplexity: 5.01
Validation set perplexity: 7.00
Average loss at step 4600: 1.624290 learning rate: 10.000000
Minibatch perplexity: 5.13
Validation set perplexity: 6.85
Average loss at step 4700: 1.636966 learning rate: 10.000000
Minibatch perplexity: 5.15
Validation set perplexity: 6.97
Average loss at step 4800: 1.642861 learning rate: 10.000000
Minibatch perplexity: 4.33
Validation set perplexity: 7.08
Average loss at step 4900: 1.642832 learning rate: 10.000000
Minibatch perplexity: 5.01
Validation set perplexity: 6.92
Average loss at step 5000: 1.618589 learning rate: 1.000000
Minibatch perplexity: 4.57
rjorting chass a is in occur ssor ext acrition partimations zero zero zero zero i
ax is may is confiction and hods infoadma that vats milital mils of ghation sing 
qy lessia many in is onla one nine eight segis sounds plan dessegoris albani day 
fpute in latives and  name canytfor